In [1]:
import org.apache.spark.sql.functions.{min, max}
import org.apache.spark.ml.recommendation.{ALS, ALSModel}
import scala.util.Random
import org.apache.spark.sql.functions.lit

In [2]:
val moviesFile = "../datasets/movielens/ml-latest-small/movies.csv"
val ratingsFile = "../datasets/movielens/ml-latest-small/ratings.csv"
val tagsFile = "../datasets/movielens/ml-latest-small/tags.csv"

moviesFile = ../datasets/movielens/ml-latest-small/movies.csv
ratingsFile = ../datasets/movielens/ml-latest-small/ratings.csv
tagsFile = ../datasets/movielens/ml-latest-small/tags.csv


../datasets/movielens/ml-latest-small/tags.csv

In [3]:
val moviesDF = spark.read.options(Map("header" -> "true", "inferSchema" -> "true")).csv(moviesFile).cache
val ratingsDF = spark.read.options(Map("header" -> "true", "inferSchema" -> "true")).csv(ratingsFile).cache
val tagsDF = spark.read.options(Map("header" -> "true", "inferSchema" -> "true")).csv(tagsFile).cache

moviesDF = [movieId: int, title: string ... 1 more field]
ratingsDF = [userId: int, movieId: int ... 2 more fields]
tagsDF = [userId: int, movieId: int ... 2 more fields]


[userId: int, movieId: int ... 2 more fields]

In [4]:
moviesDF.show

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [5]:
ratingsDF.show

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     31|   2.5|1260759144|
|     1|   1029|   3.0|1260759179|
|     1|   1061|   3.0|1260759182|
|     1|   1129|   2.0|1260759185|
|     1|   1172|   4.0|1260759205|
|     1|   1263|   2.0|1260759151|
|     1|   1287|   2.0|1260759187|
|     1|   1293|   2.0|1260759148|
|     1|   1339|   3.5|1260759125|
|     1|   1343|   2.0|1260759131|
|     1|   1371|   2.5|1260759135|
|     1|   1405|   1.0|1260759203|
|     1|   1953|   4.0|1260759191|
|     1|   2105|   4.0|1260759139|
|     1|   2150|   3.0|1260759194|
|     1|   2193|   2.0|1260759198|
|     1|   2294|   2.0|1260759108|
|     1|   2455|   2.5|1260759113|
|     1|   2968|   1.0|1260759200|
|     1|   3671|   3.0|1260759117|
+------+-------+------+----------+
only showing top 20 rows



In [6]:
tagsDF.show

+------+-------+--------------------+----------+
|userId|movieId|                 tag| timestamp|
+------+-------+--------------------+----------+
|    15|    339|sandra 'boring' b...|1138537770|
|    15|   1955|             dentist|1193435061|
|    15|   7478|            Cambodia|1170560997|
|    15|  32892|             Russian|1170626366|
|    15|  34162|         forgettable|1141391765|
|    15|  35957|               short|1141391873|
|    15|  37729|          dull story|1141391806|
|    15|  45950|          powerpoint|1169616291|
|    15| 100365|            activist|1425876220|
|    15| 100365|         documentary|1425876220|
|    15| 100365|              uganda|1425876220|
|    23|    150|          Ron Howard|1148672905|
|    68|   2174|               music|1249808064|
|    68|   2174|               weird|1249808102|
|    68|   8623|        Steve Martin|1249808497|
|    73| 107999|              action|1430799184|
|    73| 107999|               anime|1430799184|
|    73| 107999|    

In [7]:
ratingsDF.agg(min("userId"), max("userId"), min("movieId"), max("movieId"), min("rating"), max("rating")).show

+-----------+-----------+------------+------------+-----------+-----------+
|min(userId)|max(userId)|min(movieId)|max(movieId)|min(rating)|max(rating)|
+-----------+-----------+------------+------------+-----------+-----------+
|          1|        671|           1|      163949|        0.5|        5.0|
+-----------+-----------+------------+------------+-----------+-----------+



In [8]:
ratingsDF.describe().show

+-------+------------------+------------------+------------------+--------------------+
|summary|            userId|           movieId|            rating|           timestamp|
+-------+------------------+------------------+------------------+--------------------+
|  count|            100004|            100004|            100004|              100004|
|   mean| 347.0113095476181|12548.664363425463| 3.543608255669773|1.1296390869392424E9|
| stddev|195.16383797819535|26369.198968815268|1.0580641091070326|1.9168582602710962E8|
|    min|                 1|                 1|               0.5|           789652009|
|    max|               671|            163949|               5.0|          1476640644|
+-------+------------------+------------------+------------------+--------------------+



In [9]:
ratingsDF.orderBy("userId", "movieId").show

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     31|   2.5|1260759144|
|     1|   1029|   3.0|1260759179|
|     1|   1061|   3.0|1260759182|
|     1|   1129|   2.0|1260759185|
|     1|   1172|   4.0|1260759205|
|     1|   1263|   2.0|1260759151|
|     1|   1287|   2.0|1260759187|
|     1|   1293|   2.0|1260759148|
|     1|   1339|   3.5|1260759125|
|     1|   1343|   2.0|1260759131|
|     1|   1371|   2.5|1260759135|
|     1|   1405|   1.0|1260759203|
|     1|   1953|   4.0|1260759191|
|     1|   2105|   4.0|1260759139|
|     1|   2150|   3.0|1260759194|
|     1|   2193|   2.0|1260759198|
|     1|   2294|   2.0|1260759108|
|     1|   2455|   2.5|1260759113|
|     1|   2968|   1.0|1260759200|
|     1|   3671|   3.0|1260759117|
+------+-------+------+----------+
only showing top 20 rows



#### We are checking if they are duplicates in dataset

In [10]:
val distinctRatingsCount = ratingsDF.select("userId", "movieId").distinct.count
val raitingsCount = ratingsDF.count

[Stage 15:=================================================>    (184 + 1) / 200]

distinctRatingsCount = 100004
raitingsCount = 100004


100004

In [11]:
val Array(trainData, testData) = ratingsDF.randomSplit(Array(0.8, 0.2))

trainData = [userId: int, movieId: int ... 2 more fields]
testData = [userId: int, movieId: int ... 2 more fields]


[userId: int, movieId: int ... 2 more fields]

In [12]:
val trainDataCount = trainData.count()
val testDataCount = testData.count()

trainDataCount = 79793
testDataCount = 20211


20211

In [13]:
val iter = 10
val rank = 20
val model = new ALS()
    .setMaxIter(iter)
    .setRank(rank)
    .setUserCol("userId")
    .setItemCol("movieId")
    .setRatingCol("rating")
    .setPredictionCol("prediction")
    .setImplicitPrefs(false)
    .fit(trainData)


[Stage 78:===================================================>     (9 + 1) / 10]

iter = 10
rank = 20
model = als_cb78a65d56fe


als_cb78a65d56fe

In [14]:
model.setColdStartStrategy("drop")

als_cb78a65d56fe

In [15]:
model.itemFactors.show(1,false)

+---+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |features                                                                                                                                                                                                                                           |
+---+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|10 |[0.5586043, 0.09885755, -0.02311161, -0.3573222, 0.1855725, 0.32043546, 0.43681934, -0.06108938, 0.579188, 0.11571113, 0.014435331, 0.07001802, -0.087862454, -0.22608283, -0.5763294, 0.24011777, -0.012586443, 1.3518986, -0.61920166, -0.415645]|


In [16]:
model.userFactors.show(1, false)

+---+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |features                                                                                                                                                                                                                                                  |
+---+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|10 |[-0.057957187, 0.5419405, -0.098569565, -0.6136434, -0.23198462, -0.1009309, 0.49403003, 0.46374685, 1.0818579, -0.18961632, 0.24970357, 0.19307777, 0.0025974158, 0.008423488, -0.46112567, 0.46938542, 0.38866627, 1.1331686, 

In [17]:
val trainPredictions = model.transform(trainData)
trainPredictions.show

[Stage 155:====================================================>(199 + 1) / 200]+------+-------+------+----------+----------+
|userId|movieId|rating| timestamp|prediction|
+------+-------+------+----------+----------+
|   575|    148|   4.0|1012605106| 3.9289086|
|   232|    463|   4.0| 955089443| 3.8387904|
|   452|    463|   2.0| 976424451|  2.283914|
|   534|    463|   4.0| 973377486| 3.8558333|
|    30|    463|   4.0| 945277405| 3.7677555|
|   311|    463|   3.0| 898008246| 2.9460309|
|    85|    471|   3.0| 837512312| 3.1591654|
|   588|    471|   3.0| 842298526| 3.4054499|
|   126|    471|   5.0| 833287141| 3.9959264|
|   460|    471|   5.0|1072836030|  4.097976|
|   350|    471|   3.0|1011714986| 3.4426746|
|   548|    471|   4.0| 857407799|  3.737401|
|   602|    471|   3.0| 842357922| 3.7639365|
|   285|    471|   5.0| 965092130| 4.0636725|
|   440|    471|   3.0| 835337519| 3.0277364|
|    86|    471|   4.0| 848161161| 3.9639633|
|   292|    471|   3.5|1140049920|  3.766833|


trainPredictions = [userId: int, movieId: int ... 3 more fields]


[userId: int, movieId: int ... 3 more fields]

In [18]:
val testPredictions = model.transform(testData)
testPredictions.show

+------+-------+------+----------+----------+                                   
|userId|movieId|rating| timestamp|prediction|
+------+-------+------+----------+----------+
|   380|    463|   3.0| 968949106| 3.0444913|
|   242|    463|   4.0| 956685706| 3.6699631|
|   274|    471|   5.0|1074104142| 3.5783854|
|   306|    471|   3.0| 939718996| 3.4816675|
|    15|    471|   3.0|1166586067| 3.0733998|
|   358|    471|   5.0| 957479605| 3.8131983|
|   105|    471|   4.0|1085574088| 3.6024966|
|   508|    471|   4.0| 844377075| 4.0544753|
|   242|    471|   5.0| 956686752| 4.1915364|
|   468|    471|   4.0|1296197444| 3.2125635|
|   497|    496|   2.0| 939767844| 2.6722193|
|   294|    833|   2.0|1047074195| 2.4390643|
|   500|   1088|   4.0|1229098924|  3.532017|
|   306|   1088|   4.0| 939760516|  3.356336|
|    54|   1088|   5.0|1352836913| 3.5020564|
|   262|   1088|   2.0|1433938031| 1.6893109|
|   461|   1088|   3.0|1090908700| 2.5360682|
|   387|   1088|   4.0| 974790964| 3.4329736|

testPredictions = [userId: int, movieId: int ... 3 more fields]


[userId: int, movieId: int ... 3 more fields]

In [19]:
val topUserRecommendations = model.recommendForAllUsers(1).cache

topUserRecommendations = [userId: int, recommendations: array<struct<movieId:int,rating:float>>]


[userId: int, recommendations: array<struct<movieId:int,rating:float>>]

In [20]:
topUserRecommendations.orderBy($"userId").show

[Stage 211:====================================================>(198 + 1) / 200]+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1| [[121231,3.897037]]|
|     2| [[42418,4.6106668]]|
|     3| [[83411,4.5362306]]|
|     4| [[83411,6.1761436]]|
|     5|  [[8535,4.9775677]]|
|     6| [[83411,4.8929133]]|
|     7| [[83411,4.9272227]]|
|     8|  [[2563,4.8035126]]|
|     9| [[83411,4.9356546]]|
|    10| [[83411,5.1373577]]|
|    11| [[134130,5.308274]]|
|    12|  [[3879,4.9072647]]|
|    13|[[134130,4.4896884]]|
|    14|  [[83411,4.869408]]|
|    15|   [[4302,4.902035]]|
|    16|  [[1243,5.2748623]]|
|    17| [[83411,5.5341454]]|
|    18|  [[83411,4.750681]]|
|    19| [[83411,5.3707647]]|
|    20|  [[51471,4.913587]]|
+------+--------------------+
only showing top 20 rows



In [21]:
import org.apache.spark.ml.recommendation.ALSModel

case class ALSEnhanced(model: ALSModel) {
    val _spark = spark
    import _spark.implicits._
    import org.apache.spark.sql.functions.lit
    import org.apache.spark.sql.functions.desc
    
    def recommendForUser(userId: Int) = { 
        model.transform(model.itemFactors.select($"id".as("movieId"))
            .withColumn("userId", lit(userId)))
            .orderBy(desc("prediction"))
    }
}

implicit def impALStoALSEnhanced(model: ALSModel) = ALSEnhanced(model)

val recommendations = model.recommendForUser(1)

defined class ALSEnhanced
recommendations = [movieId: int, userId: int ... 1 more field]


impALStoALSEnhanced: (model: org.apache.spark.ml.recommendation.ALSModel)ALSEnhanced


[movieId: int, userId: int ... 1 more field]

In [22]:
recommendations.show

+-------+------+----------+                                                     
|movieId|userId|prediction|
+-------+------+----------+
| 121231|     1|  3.897037|
|  67504|     1| 3.7174983|
|  83359|     1| 3.7174983|
|  83411|     1| 3.7174983|
|  83318|     1| 3.7174983|
|   1243|     1| 3.6679907|
|   4755|     1| 3.6321802|
|   1264|     1| 3.6145625|
|   2105|     1|  3.470392|
|   3260|     1|  3.467227|
|   5017|     1| 3.4540367|
|    955|     1|   3.44335|
|   1734|     1| 3.4340425|
|  86377|     1|  3.423299|
|   3083|     1| 3.4232152|
|   8264|     1| 3.4165528|
|  27846|     1| 3.4085884|
| 106920|     1|  3.403481|
|  31435|     1|  3.397064|
|   3019|     1| 3.3945513|
+-------+------+----------+
only showing top 20 rows



In [23]:
recommendations.join(moviesDF, "movieId").show

+-------+------+----------+--------------------+--------------------+           
|movieId|userId|prediction|               title|              genres|
+-------+------+----------+--------------------+--------------------+
| 121231|     1|  3.897037|   It Follows (2014)|              Horror|
|  83318|     1| 3.7174983|    Goat, The (1921)|              Comedy|
|  83411|     1| 3.7174983|         Cops (1922)|              Comedy|
|  83359|     1| 3.7174983|Play House, The (...|              Comedy|
|  67504|     1| 3.7174983|Land of Silence a...|         Documentary|
|   1243|     1| 3.6679907|Rosencrantz and G...|        Comedy|Drama|
|   4755|     1| 3.6321802|Wish Upon a Star ...|              Comedy|
|   1264|     1| 3.6145625|         Diva (1981)|Action|Drama|Myst...|
|   2105|     1|  3.470392|         Tron (1982)|Action|Adventure|...|
|   3260|     1|  3.467227|  Howards End (1992)|               Drama|
|   5017|     1| 3.4540367|Big Heat, The (1953)|     Drama|Film-Noir|
|    955|

### Evaluations

In [24]:
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.Row

def evaluatePredictions(predictions: Dataset[Row]): Double = {
    import org.apache.spark.ml.evaluation.RegressionEvaluator
    
    val evaluator = new RegressionEvaluator()
        .setMetricName("rmse")
        .setLabelCol("rating")
        .setPredictionCol("prediction")
    
    evaluator.evaluate(predictions)
}

evaluatePredictions: (predictions: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row])Double


* trainData performance

In [25]:
val rmse = evaluatePredictions(trainPredictions)
println(s"RMSE: $rmse")

[Stage 329:====================================================>(198 + 1) / 200]RMSE: 0.5122725666179361


rmse = 0.5122725666179361


0.5122725666179361

In [26]:
val rmse = evaluatePredictions(testPredictions)
println(s"RMSE: $rmse")

RMSE: 0.9170539467582317                                                        


rmse = 0.9170539467582317


0.9170539467582317

* testData performance

# Test hyperparameters 

In [33]:
val ranks = Seq(10, 100)
val regParams = Seq(2, 0.05, 0.0001)
val alphas = Seq(0.01, 10)
val iters = Seq(10, 20)

ranks = List(10, 100)
regParams = List(2.0, 0.05, 1.0E-4)
alphas = List(0.01, 10.0)
iters = List(10, 20)


List(10, 20)

In [34]:
import org.apache.spark.sql.functions.udf
val isNaN = udf { v: Double => v.isNaN }

isNaN = UserDefinedFunction(<function1>,BooleanType,Some(List(DoubleType)))


UserDefinedFunction(<function1>,BooleanType,Some(List(DoubleType)))

In [38]:
for (rank <- ranks; regParam <- regParams; alpha <- alphas) {
    val model = new ALS()
        .setMaxIter(iter)
        .setRank(rank)
        .setAlpha(alpha)
        .setRegParam(regParam)
        .setUserCol("userId")
        .setItemCol("movieId")
        .setRatingCol("rating")
        .setPredictionCol("prediction")
        .setImplicitPrefs(false)
        .fit(trainData)
        
    model.setColdStartStrategy("drop")
        
    val trainPredictions = model.transform(trainData)
    val testPredictions = model.transform(testData)
    
    val trainRMSE = evaluatePredictions(trainPredictions)
    val testRMSE = evaluatePredictions(testPredictions)
    
    println(s"Rank: ${rank}, RegParam: ${regParam}, Alpha: ${alpha}")
    println(s"\t Train RMSE: ${trainRMSE}")
    println(s"\t Test RMSE: ${testRMSE}")
    println
}

Rank: 10, RegParam: 2.0, Alpha: 0.01                                            
	 Train RMSE: 2.1545798738250093
	 Test RMSE: 2.1832116656462577

Rank: 10, RegParam: 2.0, Alpha: 10.0                                            
	 Train RMSE: 2.1545798738250093
	 Test RMSE: 2.1832116656462577

Rank: 10, RegParam: 0.05, Alpha: 0.01                                           
	 Train RMSE: 0.5170003376388996
	 Test RMSE: 0.9881429612293389

Rank: 10, RegParam: 0.05, Alpha: 10.0                                           
	 Train RMSE: 0.5170003376388996
	 Test RMSE: 0.9881429612293389

Rank: 10, RegParam: 1.0E-4, Alpha: 0.01                                         
	 Train RMSE: 0.49754727135166815
	 Test RMSE: 1.5290804178416728

Rank: 10, RegParam: 1.0E-4, Alpha: 10.0                                         
	 Train RMSE: 0.49754727135166815
	 Test RMSE: 1.5290804178416728

Rank: 100, RegParam: 2.0, Alpha: 0.01                                           
	 Train RMSE: 2.1545798728538643
	 

lastException: Throwable = null


In [40]:
trainData.filter(isNaN($"rating")).count()

0